# Studying Geostrophic Events in the Southern Ocean with SWOT Data: Analyzing Upwelling and Downwelling events influenced by the Ekman Transport to Observe Impacts on Carbon Uptake

This Jupyter Notebook contains the code necessary to recreate all of my figures and work for my study on how geostrophic events like upwelling and downwelling, particularly those influenced by the Ekman Transport, impact carbon uptake in the Southern Ocean. This project was mentored by the Dr. Andrew Thompson, director of Caltech's Linde and Maxine Center for Global Environmental Science, and it was completed in 2024.

See images of figures from this notebook here: https://drive.google.com/drive/folders/1KyChSgVsjj3VhcA9_azXk9evaQ11TzWT?usp=sharing

See the final presentation here: https://docs.google.com/presentation/d/1Rq1q147vMwHig1qE8VyV0LJPZu_svA6IS9iNE0U7Q8U/edit?usp=sharing

## Relevant Information

### Background Information

The Southern Ocean is a major carbon sink, accounting for approximately 40% of the ocean's global uptake of anthropogenic carbon dioxide. Upwelling is the process of cold, nutrient-rich water rising from deep into the ocean to the surface, whereas downwelling is the process of surface water moving downward. Both contribute to carbon uptake, as downwelling sequesters carbon rich surface water deeper into the ocean, while upwelling pushes the nutrients necessary for organisms like phytoplankton to perform photosythesis and further sequester carbon. The Ekman transport, a fundamental physical driver of ocean circulation is a primary cause for wind induced upwelling and downwelling in many coastal areas, and facilitates both the biological and physical processes responsible for moving and storing carbon within the ocean system.

### Relevant Papers

*Fu et al. 2024* "The Surface Water and Ocean Topography Mission: A Breakthrough in Radar Remote Sensing of the Ocean and Land Surface Water"

**Link:** https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2023GL107652

*Morrison et al. 2015* "Upwelling in the Southern Ocean"

**Link:** https://www.researchgate.net/publication/270508719_Upwelling_in_the_Southern_Ocean

## Code (work, figures, etc.)

### Imports

In [ ]:
%matplotlib qt5


import scipy
import scipy.linalg
import scipy.interpolate as interp

import numpy as np
from numpy.random import default_rng

import cartopy.crs as ccrs
import cartopy

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D

from skimage import exposure
from skimage.io import imsave, imread

from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm.notebook import trange, tqdm

import geopandas as gpd
import pandas as pd
import geoviews as gv
import hvplot.pandas

import pyproj
from pyproj import Proj, transform
from pyproj import Transformer


from ipywidgets import IntProgress
from IPython.display import display

import rasterio as rio
from rasterio.features import rasterize
from rasterio.session import AWSSession
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.warp import Resampling as resample

from osgeo import gdal
from osgeo import ogr

import pygmt
from sklearn.neighbors import BallTree
import earthaccess
import h5netcdf
import xarray as xr
import rioxarray
import pystac_client
import intake
import boto3
import statsmodels.formula.api as smf
from shapely.geometry.polygon import Polygon, Point
import gc
import pytz
from pathlib import Path
import pprint
import dask
import os
import copy

import warnings
warnings.filterwarnings('ignore')

### SWOT Data Management (if you HAVE authentification for earthaccess database)

If you only have access to downloaded files, skip this and proceed to next section.

In [ ]:
# authenticate
os.environ["AWS_REQUEST_PAYER"] = "requester"
aws_session = AWSSession(boto3.Session(), requester_pays=True)

In [ ]:
# authenticate for NASA data
auth = earthaccess.login(strategy="netrc")

if not auth.authenticated:
    # Ask for credentials and persist them in a .netrc file
    auth.login(strategy="interactive", persist=True)

In [ ]:
bbox = (150,-54,160,-52) 

# change start + end date accordingly

start_dt = '2024-01-19'
end_dt = '2024-01-21'

In [ ]:
import earthaccess

# gather all files from terra + aqua on same day as landsat image
results = earthaccess.search_data(
    short_name='SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc',
    bounding_box=bbox,
    # searches of day only
    temporal=(start_dt, end_dt)
)
print (f'{len(results)} TOTAL granules')

In [ ]:
# files = earthaccess.open(results)
# ds = xr.open_mfdataset(files)

import xarray as xr

# to image 1 day
ds = xr.open_dataset(earthaccess.open(results[0:2])[0], group='left',engine='h5netcdf')

# mask for lat
mask = (ds.latitude >= -54) & (ds.latitude <= -52)
print(mask)

# mask to slice dataset
ds_sliced = ds.where(mask, drop=True)

# flatten lat and lon
lat = ds_sliced.latitude.values.ravel()
lon = ds_sliced.longitude.values.ravel()
ssh_karin_2 = ds_sliced.ssh_karin_2.values.ravel()

# mask to filter out non-finite values
mask = np.isfinite(lat) & np.isfinite(lon) & np.isfinite(ssh_karin_2)

# mask data
lat = lat[mask]
lon = lon[mask]
ssh_karin_2 = ssh_karin_2[mask]

In [ ]:
# open dataset
ds = xr.open_mfdataset(earthaccess.open(results[0:7]), combine='nested', concat_dim="num_lines", decode_times=False, group='left',engine='h5netcdf')
ds

In [ ]:
# mask for lat range
mask = (ds.latitude >= -54) & (ds.latitude <= -52) & (ds.longitude >= 150) & (ds.longitude <= 160)
mask = mask.compute()
ds_sliced = ds.where(mask, drop=True)

# flatten lat and lon
lat = ds_sliced.latitude.values.ravel()
lon = ds_sliced.longitude.values.ravel()
ssh_karin_2 = ds_sliced.ssh_karin_2.values.ravel()

# mask to filter out non-finite values
mask = np.isfinite(lat) & np.isfinite(lon) & np.isfinite(ssh_karin_2)

In [ ]:
# plot cleaned data
plt.figure(figsize=(8, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

# set lat and lon limits
# pcolormesh to plot the cleaned data
p = ax.scatter(
    ds.longitude, ds.latitude, c=ds.ssh_karin_2,
    transform=ccrs.PlateCarree(), vmin=-30, vmax=-20, cmap='coolwarm', s=1
)

# colorbar
plt.colorbar(p, ax=ax, orientation='vertical', pad=0.02, aspect=50)

# coastlines
ax.coastlines()

# axis ticks
import matplotlib.ticker as ticker
import numpy as np

ax.set_yticks(np.arange(-80, 80.1, 20))
ax.set_xticks(np.arange(-120, -79.9, 20))

# display plot
plt.show()

In [ ]:
# retrieves granule from desired day
karin_results = earthaccess.search_data(short_name = 'SWOT_L2_LR_SSH_EXPERT_2.0', 
                                        bounding_box=bbox, 
                                        temporal=(start_dt, end_dt))

In [ ]:
# open granules and load into xarray dataset
ds = xr.open_mfdataset(earthaccess.open(karin_results[0:12]), combine='nested', concat_dim="num_lines", decode_times=False, engine='h5netcdf')
ds

ds['ssh_karin_2_corrected'] = ds.ssh_karin_2 + ds.height_cor_xover
ds['ssha_karin_corrected'] = ds.ssha_karin + ds.height_cor_xover
ds.ssh_karin_2_corrected

# print the keys of the dataset object to see the available variables
print(ds.keys())

In [ ]:
ds = xr.open_dataset(earthaccess.open(results[0:2])[0], group='left',engine='h5netcdf')


plt.figure(figsize=(9, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()

# Set the latitude and longitude limits
# ax.set_extent([-110, -99, -75.44, -71], crs=ccrs.PlateCarree())
ax.set_extent([-104.1, -100.9, -75.29, -74.2], crs=ccrs.PlateCarree())

ds.ssh_karin_2_corrected.plot.pcolormesh(
 ax=ax, transform=ccrs.PlateCarree(), x="longitude", y="latitude", vmin=-31, vmax=-27, cmap='coolwarm', add_colorbar=True
) # vmin=-12, vmax=5,
ax.coastlines()

### SWOT Data Management (if you DO NOT have authentification for earthaccess database)

In [ ]:
# open file
file2read = netCDF4.Dataset('SWOT_L2_LR_SSH_Unsmoothed_008_561_20240103T065942_20240103T075109_PIC0_01.nc','r')

In [ ]:
# gathering groups + variables

var = file2read.groups

left = file2read.groups['left']
right = file2read.groups['right']

leftvar = left.variables
rightvar = right.variables

leftlat = left['latitude'][0:1]
rightlat = right['latitude'] [0:1]

leftlon = left['longitude']
rightlon = right ['longitude']

file2read.close()

# lat -55 to -52, lon 150 to 160
# vars: SSH_karin, SSHA_karin, height_cor_xover, latitude, longitude

In [ ]:
# printing index (for easy copy paste later)

# print(var)
# print(left)
# print(right)
# print(leftvar)
# print(rightvar)
# print(leftlat)
# print(rightlat)
# print(leftlon)
# print(rightlon)

In [ ]:
# plotting

# create plot
fig, axes = plt.subplots(1,1, figsize=(2, 10))

# add axis ticks
plt.setp(axes, xticks=[0, 0.5, 1], xticklabels=['-54', '-53', '-52'], 
         yticks=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
         yticklabels=['150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160'])

plt.show()

### SWOT Expert Data Compiling + Plots

Plots a single SWOT Expert Pass, visualizing Sea Surface Height (SSH) as well as Sea Surface Height Anomaly (SSH Anomaly), can change latitude and longitude range (provided necessary files are accessible) to preference.

In [ ]:
# open file
# data = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc','r')

# pass 310: SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc
# pass 127: 

In [ ]:
# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

In [ ]:
# open file
data = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc','r')

# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssh = data.variables ['ssh_karin'] [:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat >= -54) & (lat <= -52) & (lon >= 150) & (lon <= 160))[0]
lat_155_ex = lat[lim]
lon_155_ex = lon[lim]
ssh_155_ex = ssh[lim] + hcorxover[lim]
ssha_155_ex = ssha[lim] + hcorxover[lim]

# plot ssh
plt.figure(45)
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssh_155_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure(46)
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssha_155_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height Anomaly')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

In [ ]:
# create an empty list to store data from each file
lat_all = []
lon_all = []
ssh_all = []
ssha_all = []
hcorxover_all = []

# all file names
file_names = ['SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc', 
              'SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PIC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc']

# loop each file
for file_name in file_names:
    data = netCDF4.Dataset(file_name, 'r')
    lat = data.variables['latitude'][:]
    lon = data.variables['longitude'][:]
    ssh = data.variables['ssh_karin'][:]
    ssha = data.variables['ssha_karin'][:]
    hcorxover = data.variables['height_cor_xover'][:]

    lat_all.extend(lat)
    lon_all.extend(lon)
    ssh_all.extend(ssh)
    ssha_all.extend(ssha)
    hcorxover_all.extend(hcorxover)

# convert to numpy arrays
# lat_all = np.array(lat_all)
# lon_all = np.array(lon_all)

# set limits
lim = np.where((lat_all >= -54) & (lat_all <= -52) & (lon_all >= 150) & (lon_all <= 160))[0]
lat_ex = lat_all[lim]
lon_ex = lon_all[lim]
ssh_ex = ssh_all[lim] + hcorxover_all[lim]
ssha_ex = ssha_all[lim] + hcorxover_all[lim]

# set limits
# lim = np.where((lat_all >= -54) & (lat_all <= -52) & (lon_all >= 150) & (lon_all <= 160))[0]
# lat_ex = lat_all[lim]
# lon_ex = lon_all[lim]
# ssh_ex = np.array(ssh_all)[lim] 
# ssha_ex = np.array(ssha_all)[lim]

# plot fig
plt.figure()
plt.scatter(lon_ex, lat_ex, s=8, c=ssh_ex, cmap='plasma', marker='o')
plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# sanity check
a = len(lat_all)
b = len(lon_all)

print(a)
print(b)

In [ ]:
plt.figure()
plt.plot(lon_ex, lat_ex)
plt.figure()
plt.plot(ssh_ex)

c = lon_ex.shape
d = lat_ex.shape
e = ssh_ex.shape

print(c)
print(d)
print(e)

print(lim)
len(lim)

In [ ]:
# open file
data = netCDF4.Dataset('enter txt','r')

# pass 310: SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc
# pass 127: 

In [ ]:
# set limits, change lat lon range to preference
lim = np.where((lat >= -54) & (lat <= -52) & (lon >= 150) & (lon <= 160))[0]
lat_310_ex = lat[lim]
lon_310_ex = lon[lim]
ssh_310_ex = ssh[lim] + hcorxover[lim]
ssha_310_ex = ssha[lim] + hcorxover[lim]

# plot ssh
plt.figure(45)
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssh_310_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# plot ssh anomaly
plt.figure(46)
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssha_310_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height Anomaly')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# create an empty list to store data from each file
lat_all = []
lon_all = []
ssh_all = []
ssha_all = []
hcorxover_all = []

# all file names
file_names = ['SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc', 
              'SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PICO_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc']

# loop each file
for file_name in file_names:
    data = netCDF4.Dataset(file_name, 'r')
    lat = data.variables['latitude'][:]
    lon = data.variables['longitude'][:]
    ssh = data.variables['ssh_karin'][:]
    ssha = data.variables['ssha_karin'][:]
    hcorxover = data.variables['height_cor_xover'][:]

    lat_all.extend(lat)
    lon_all.extend(lon)
    ssh_all.extend(ssh + hcorxover)
    ssha_all.extend(ssha + hcorxover)
    hcorxover_all.extend(hcorxover)

# plot data
plt.figure()
plt.scatter(lon_all, lat_all, s=8, c=ssh_all, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

### SWOT Basic Data Compiling + Plots

Plots a single SWOT Basic Pass, visualizing Sea Surface Height (SSH) as well as Sea Surface Height Anomaly (SSH Anomaly), can change latitude and longitude range (provided necessary files are accessible) to preference.

In [ ]:
# open file
data = netCDF4.Dataset('SWOT_L2_LR_SSH_Basic_009_310_20240115T043134_20240115T052302_PGC0_01.nc','r')

In [ ]:
# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssh = data.variables['ssh_karin'][:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

In [ ]:
# set limits, change lat lon range to preference
lim = np.where((lat >= -55) & (lat <= -52) & (lon >= 150) & (lon <= 160))[0]
lat_310_ex = lat[lim]
lon_310_ex = lon[lim]
ssh_310_ex = ssh[lim] + hcorxover[lim]
ssha_310_ex = ssha[lim] + hcorxover[lim]

# plot ssh
plt.figure(45)
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssh_310_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# plot ssh anomaly
plt.figure(46)
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssha_310_ex, cmap='viridis', marker='o')
plt.colorbar()
plt.title('Sea Surface Height Anomaly')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

### SWOT Full Expert Data Compilation + Plots

Plots all SWOT Expert Passes within a certain latitude and longitude Range on one plot to visualize Sea Surface Height (SSH) and Sea Surface Height Anomaly (SSH Anomaly), can change latitude and longitude range (provided necessary files are accessible) to preference.

Granted, SWOT Passes are typically angled and thus ascending and descending passes can overlap and cover one another, so refer to sub-sections to view ascending/descending SWOT Data separately for full visibility.

#### SWOT Full Expert Data Plots (may cover one another)

**ALL FILE NAMES FOR EASY PASTING + SANITY (IF USING SAME FILES AS ORIGINAL)**

SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PIC0_01.nc SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc

In [ ]:
# pass no. 127

# open file 
data_127 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc','r')

# gather variables
lat_127 = data_127.variables['latitude'][:]
lon_127 = data_127.variables['longitude'][:]
ssh_127 = data_127.variables ['ssh_karin'] [:]
ssha_127 = data_127.variables['ssha_karin'][:]
hcorxover_127 = data_127.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_127 >= -54) & (lat_127 <= -52) & (lon_127 >= 150) & (lon_127 <= 160))[0]
lat_127_ex = lat_127[lim]
lon_127_ex = lon_127[lim]
ssh_127_ex = ssh_127[lim] + hcorxover_127[lim]
ssha_127_ex = ssha_127[lim] + hcorxover_127[lim]

In [ ]:
# pass no. 155

# open file
data_155 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc','r')

# gather variables
lat_155 = data_155.variables['latitude'][:]
lon_155 = data_155.variables['longitude'][:]
ssh_155 = data_155.variables['ssh_karin'][:]
ssha_155 = data_155.variables['ssha_karin'][:]
hcorxover_155 = data_155.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_155 >= -54) & (lat_155 <= -52) & (lon_155 >= 150) & (lon_155 <= 160))[0]
lat_155_ex = lat_155[lim]
lon_155_ex = lon_155[lim]
ssh_155_ex = ssh_155[lim] + hcorxover_155[lim]
ssha_155_ex = ssha_155[lim] + hcorxover_155[lim]

In [ ]:
# pass no. 183

# open file
data_183 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc','r')

# gather variables
lat_183 = data_183.variables['latitude'][:]
lon_183 = data_183.variables['longitude'][:]
ssh_183 = data_183.variables['ssh_karin'][:]
ssha_183 = data_183.variables['ssha_karin'][:]
hcorxover_183 = data_183.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_183 >= -54) & (lat_183 <= -52) & (lon_183 >= 150) & (lon_183 <= 160))[0]
lat_183_ex = lat_183[lim]
lon_183_ex = lon_183[lim]
ssh_183_ex = ssh_183[lim] + hcorxover_183[lim]
ssha_183_ex = ssha_183[lim] + hcorxover_183[lim]

In [ ]:
# pass no. 254

# open file
data_254 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc','r')

# gather variables
lat_254 = data_254.variables['latitude'][:]
lon_254 = data_254.variables['longitude'][:]
ssh_254 = data_254.variables['ssh_karin'][:]
ssha_254 = data_254.variables['ssha_karin'][:]
hcorxover_254 = data_254.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_254 >= -54) & (lat_254 <= -52) & (lon_254 >= 150) & (lon_254 <= 160))[0]
lat_254_ex = lat_254[lim]
lon_254_ex = lon_254[lim]
ssh_254_ex = ssh_254[lim] + hcorxover_254[lim]
ssha_254_ex = ssha_254[lim] + hcorxover_254[lim]

In [ ]:
# pass no. 282

# open file
data_282 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc','r')

# gather variables
lat_282 = data_282.variables['latitude'][:]
lon_282 = data_282.variables['longitude'][:]
ssh_282 = data_282.variables['ssh_karin'][:]
ssha_282 = data_282.variables['ssha_karin'][:]
hcorxover_282 = data_282.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_282 >= -54) & (lat_282 <= -52) & (lon_282 >= 150) & (lon_282 <= 160))[0]
lat_282_ex = lat_282[lim]
lon_282_ex = lon_282[lim]
ssh_282_ex = ssh_282[lim] + hcorxover_282[lim]
ssha_282_ex = ssha_282[lim] + hcorxover_282[lim]

In [ ]:
# pass no. 310

# open file
data_310 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc','r')

# gather variables
lat_310 = data_310.variables['latitude'][:]
lon_310 = data_310.variables['longitude'][:]
ssh_310 = data_310.variables['ssh_karin'][:]
ssha_310 = data_310.variables['ssha_karin'][:]
hcorxover_310 = data_310.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_310 >= -54) & (lat_310 <= -52) & (lon_310 >= 150) & (lon_310 <= 160))[0]
lat_310_ex = lat_310[lim]
lon_310_ex = lon_310[lim]
ssh_310_ex = ssh_310[lim] + hcorxover_310[lim]
ssha_310_ex = ssha_310[lim] + hcorxover_310[lim]

In [ ]:
# pass no 338

# open file
data_338 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PIC0_01.nc','r')

# gather variables
lat_338 = data_338.variables['latitude'][:]
lon_338 = data_338.variables['longitude'][:]
ssh_338 = data_338.variables['ssh_karin'][:]
ssha_338 = data_338.variables['ssha_karin'][:]
hcorxover_338 = data_338.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_338 >= -54) & (lat_338 <= -52) & (lon_338 >= 150) & (lon_338 <= 160))[0]
lat_338_ex = lat_338[lim]
lon_338_ex = lon_338[lim]
ssh_338_ex = ssh_338[lim] + hcorxover_338[lim]
ssha_338_ex = ssha_338[lim] + hcorxover_338[lim]

In [ ]:
# pass no. 405

# open file
data_405 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc','r')

# gather variables
lat_405 = data_405.variables['latitude'][:]
lon_405 = data_405.variables['longitude'][:]
ssh_405 = data_405.variables['ssh_karin'][:]
ssha_405 = data_405.variables['ssha_karin'][:]
hcorxover_405 = data_405.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_405 >= -54) & (lat_405 <= -52) & (lon_405 >= 150) & (lon_405 <= 160))[0]
lat_405_ex = lat_405[lim]
lon_405_ex = lon_405[lim]
ssh_405_ex = ssh_405[lim] + hcorxover_405[lim]
ssha_405_ex = ssha_405[lim] + hcorxover_405[lim]

In [ ]:
# pass no. 433

# open file
data_433 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc','r')

# gather variables
lat_433 = data_433.variables['latitude'][:]
lon_433 = data_433.variables['longitude'][:]
ssh_433 = data_433.variables['ssh_karin'][:]
ssha_433 = data_433.variables['ssha_karin'][:]
hcorxover_433 = data_433.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_433 >= -54) & (lat_433 <= -52) & (lon_433 >= 150) & (lon_433 <= 160))[0]
lat_433_ex = lat_433[lim]
lon_433_ex = lon_433[lim]
ssh_433_ex = ssh_433[lim] + hcorxover_433[lim]
ssha_433_ex = ssha_433[lim] + hcorxover_433[lim]

In [ ]:
# pass no. 461

# open file
data_461 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc','r')

# gather variables
lat_461 = data_461.variables['latitude'][:]
lon_461 = data_461.variables['longitude'][:]
ssh_461 = data_461.variables['ssh_karin'][:]
ssha_461 = data_461.variables['ssha_karin'][:]
hcorxover_461 = data_461.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_461 >= -54) & (lat_461 <= -52) & (lon_461 >= 150) & (lon_461 <= 160))[0]
lat_461_ex = lat_461[lim]
lon_461_ex = lon_461[lim]
ssh_461_ex = ssh_461[lim] + hcorxover_461[lim]
ssha_461_ex = ssha_461[lim] + hcorxover_461[lim]

In [ ]:
# pass no. 489

# open file
data_489 = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc','r')

# gather variables
lat_489 = data_489.variables['latitude'][:]
lon_489 = data_489.variables['longitude'][:]
ssh_489 = data_489.variables['ssh_karin'][:]
ssha_489 = data_489.variables['ssha_karin'][:]
hcorxover_489 = data_489.variables['height_cor_xover'][:]

# set limits
lim = np.where((lat_489 >= -54) & (lat_489 <= -52) & (lon_489 >= 150) & (lon_489 <= 160))[0]
lat_489_ex = lat_489[lim]
lon_489_ex = lon_489[lim]
ssh_489_ex = ssh_489[lim] + hcorxover_489[lim]
ssha_489_ex = ssha_489[lim] + hcorxover_489[lim]

In [ ]:
# plotting

# plot ssh
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssh_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssh_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssh_183_ex, cmap='viridis', marker='o')
plt.scatter(lon_254_ex, lat_254_ex, s=8, c=ssh_254_ex, cmap='viridis', marker='o')
plt.scatter(lon_282_ex, lat_282_ex, s=8, c=ssh_282_ex, cmap='viridis', marker='o')
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssh_310_ex, cmap='viridis', marker='o')
plt.scatter(lon_338_ex, lat_338_ex, s=8, c=ssh_338_ex, cmap='viridis', marker='o')
plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssh_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssh_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssh_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssh_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssha_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssha_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssha_183_ex, cmap='viridis', marker='o')
plt.scatter(lon_254_ex, lat_254_ex, s=8, c=ssha_254_ex, cmap='viridis', marker='o')
plt.scatter(lon_282_ex, lat_282_ex, s=8, c=ssha_282_ex, cmap='viridis', marker='o')
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssha_310_ex, cmap='viridis', marker='o')
plt.scatter(lon_338_ex, lat_338_ex, s=8, c=ssha_338_ex, cmap='viridis', marker='o')
plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssha_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssha_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssha_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssha_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height Anomaly')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

#### Ascending Plots

In [ ]:
# plot ssh
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssh_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssh_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssh_183_ex, cmap='viridis', marker='o')
plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssh_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssh_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssh_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssh_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssha_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssha_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssha_183_ex, cmap='viridis', marker='o')
plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssha_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssha_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssha_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssha_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height Anomaly (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

In [ ]:
# plot ssh
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssh_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssh_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssh_183_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure()

plt.scatter(lon_127_ex, lat_127_ex, s=8, c=ssha_127_ex, cmap='viridis', marker='o')
plt.scatter(lon_155_ex, lat_155_ex, s=8, c=ssha_155_ex, cmap='viridis', marker='o')
plt.scatter(lon_183_ex, lat_183_ex, s=8, c=ssha_183_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height Anomaly (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

In [ ]:
# plot ssh
plt.figure()

plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssh_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssh_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssh_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssh_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure()

plt.scatter(lon_405_ex, lat_405_ex, s=8, c=ssha_405_ex, cmap='viridis', marker='o')
plt.scatter(lon_433_ex, lat_433_ex, s=8, c=ssha_433_ex, cmap='viridis', marker='o')
plt.scatter(lon_461_ex, lat_461_ex, s=8, c=ssha_461_ex, cmap='viridis', marker='o')
plt.scatter(lon_489_ex, lat_489_ex, s=8, c=ssha_489_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height Anomaly (Ascending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

#### Descending Plots

In [ ]:
# plot ssh
plt.figure()

plt.scatter(lon_254_ex, lat_254_ex, s=8, c=ssh_254_ex, cmap='viridis', marker='o')
plt.scatter(lon_282_ex, lat_282_ex, s=8, c=ssh_282_ex, cmap='viridis', marker='o')
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssh_310_ex, cmap='viridis', marker='o')
plt.scatter(lon_338_ex, lat_338_ex, s=8, c=ssh_338_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height (Descending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

# plot ssh anomaly
plt.figure()

plt.scatter(lon_254_ex, lat_254_ex, s=8, c=ssha_254_ex, cmap='viridis', marker='o')
plt.scatter(lon_282_ex, lat_282_ex, s=8, c=ssha_282_ex, cmap='viridis', marker='o')
plt.scatter(lon_310_ex, lat_310_ex, s=8, c=ssha_310_ex, cmap='viridis', marker='o')
plt.scatter(lon_338_ex, lat_338_ex, s=8, c=ssha_338_ex, cmap='viridis', marker='o')

plt.colorbar()
plt.title('Sea Surface Height Anomaly (Descending)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.show()

### SWOT Sea Surface Height Interpolation + Velocity Plots

In [ ]:
# open file
data = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc','r')

# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssh = data.variables ['ssh_karin'] [:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

ssh_1 = np.add(ssh, hcorxover)
ssha_1 = np.add(ssha, hcorxover)

ssh_mask = ssh_1 <= 200000000
ssha_mask = ssha_1 <= 20000000

ssh_new = ssh_1[ssh_mask]
ssha_new = ssha_1[ssha_mask]

a = len(lat)
b = len(lon)
c = len(ssh_new)

# sanity check
print(a,b,c)

In [ ]:
# constants
g = 9.81  # acceleration due to gravity (m/s^2)
omega = 7.2921159e-5  # angular velocity of Earth (rad/s)

min_lon = 153
max_lon = 153.1
min_lat = -53.5
max_lat = -53.4  

# function to create mesh grid
def mesh(min_lon, max_lon, min_lat, max_lat, n):
    return np.meshgrid(np.linspace(min_lon, max_lon, n), np.linspace(min_lat, max_lat, n))

# sparse input mesh for SSH data
N_sparse = 38

# define the ranges for lat and lon
lon_range = np.arange(152, 155, 0.1)
lat_range = np.arange(-54.5, -51, 0.1)

# create figure for plot
plt.figure(figsize=(15, 10))

for i, lon in enumerate(lon_range):
    for j, lat in enumerate(lat_range):
        # create mesh grid for current 0.1 by 0.1 degree area
        x_sparse, y_sparse = mesh(lon, lon + 0.1, lat, lat + 0.1, N_sparse)

        # interpolate SSH onto dense grid
        grid_ssh = interp.griddata((lon_filt.flatten(), lat_filt.flatten()), ssh_filt.flatten(),
                                   (x_sparse, y_sparse), method='nearest')

        # calculate gradients for geostrophic velocity
        dSSH_dy, dSSH_dx = np.gradient(grid_ssh)

        # calculate Coriolis parameter
        f = 2 * omega * np.sin(np.radians(y_sparse))

        # calculate velocities
        U = -g / f * dSSH_dy
        V = g / f * dSSH_dx  # corrected to use dSSH_dx for V

        # create a subplot for each 0.1 by 0.1 degree area
        plt.subplot(len(lon_range), len(lat_range), i * len(lat_range) + j + 1)
        plt.quiver(x_sparse, y_sparse, U, V)
        plt.title(f'Velocity at Lon: {lon:.1f}, Lat: {lat:.1f}')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.axis('equal')

# adjust layout + show plots
plt.tight_layout()
plt.show()

In [ ]:
def mesh(n):
    minval_lon = 153
    maxval_lon = 153.1
    minval_lat = -53.5
    maxval_lat = -53.4
    return np.meshgrid(np.linspace(minval_lon, maxval_lon, n),
                       np.linspace(minval_lat, maxval_lat, n))
    
# Function to create mesh grid
# def mesh(min_lon, max_lon, min_lat, max_lat, n):
#return np.meshgrid(np.linspace(min_lon, max_lon, n),
                       # np.linspace(min_lat, max_lat, n))

# Sparse input mesh for SSH data
N_sparse = 38

In [ ]:
# create an empty list to store data from each file
lat_all = []
lon_all = []
ssh_all = []
ssha_all = []
hcorxover_all = []

# all file names
file_names = ['SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc', 
              'SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PIC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc']

# pass 155/433

In [ ]:
# loop each file
for file_name in file_names:
    data = netCDF4.Dataset(file_name, 'r')
    lat = data.variables['latitude'][:]
    lon = data.variables['longitude'][:]
    ssh = data.variables['ssh_karin'][:]
    ssha = data.variables['ssha_karin'][:]
    hcorxover = data.variables['height_cor_xover'][:]

    lat_all.extend(lat)
    lon_all.extend(lon)
    ssh_all.extend(ssh)
    ssha_all.extend(ssha)
    hcorxover_all.extend(hcorxover)

# convert to array
longitude = np.array(lon_all)
latitude = np.array(lat_all)
# ssh = np.array(ssh_all + hcorxover_all)
# ssha = np.array(ssha_all + hcorxover_all)
ssh = np.add(ssh_all, hcorxover_all)
ssha = np.add(ssha_all, hcorxover_all)

In [ ]:
ssh_mask = ssh <= 200000000
ssha_mask = ssha <= 20000000

ssh_new = ssh[ssh_mask]
ssha_new = ssha[ssha_mask]

print(ssh_new)
print(ssha_new)

In [ ]:
# set limits + chunk data

# set lon and lat limits 
lon_min, lon_max = 153, 153.1
lat_min, lat_max = -53.5, -53.4

# create mask
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max) & (ssh <= 200000000) & (ssha <= 20000000) 

# apply mask to slice data
lon_1 = longitude[mask]
lat_1 = latitude[mask]
ssh_1 = ssh[mask]
ssha_1 = ssha[mask]

In [ ]:
# 156-167

# set lon and lat limits 
lon_min, lon_max = 156, 157
lat_min, lat_max = -55, -52 

# create mask
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max)

# apply mask to slice data
lon_2 = longitude[mask]
lat_2 = latitude[mask]
ssh_2 = ssh[mask]
ssha_2 = ssha[mask]

In [ ]:
# 157-158

# set lon and lat limits 
lon_min, lon_max = 157, 158
lat_min, lat_max = -55, -52 

# create mask
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max)

# apply mask to slice data
lon_3 = longitude[mask]
lat_3 = latitude[mask]
ssh_3 = ssh[mask]
ssha_3 = ssha[mask]

In [ ]:
# 158-159

# set lon and lat limits 
lon_min, lon_max = 158, 159
lat_min, lat_max = -55, -52 

# create mask
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max)

# apply mask to slice data
lon_4 = longitude[mask]
lat_4 = latitude[mask]
ssh_4 = ssh[mask]
ssha_4 = ssha[mask]

In [ ]:
# 159-160

# set lon and lat limits 
lon_min, lon_max = 159, 160
lat_min, lat_max = -55, -52 

# create mask
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max)

# apply mask to slice data
lon_5 = longitude[mask]
lat_5 = latitude[mask]
ssh_5 = ssh[mask]
ssha_5 = ssha[mask]

In [ ]:
a = len(ssh_5)
b = len(ssh_4)
c = len(ssh_3)
d = len(ssh_2)
e = len(ssh_1)

total = a + b + c + d + e

print(total)

In [ ]:
# plot SSH + velocity field

# constants
g = 9.81  # acceleration due to gravity (m/s^2)
omega = 7.2921159e-5  # angular velocity of Earth (rad/s)

# aux function for mesh generation
def mesh(n):
    minval_lon = 153
    maxval_lon = 153.1
    minval_lat = -53.5
    maxval_lat = -53.4
    return np.meshgrid(np.linspace(minval_lon, maxval_lon, n),
                       np.linspace(minval_lat, maxval_lat, n))

# sparse input mesh for SSH data
N_sparse = 38
x_sparse, y_sparse = mesh(N_sparse)

# interpolate SSH onto dense grid
grid_ssh = interp.griddata((lon_1.flatten(), lat_1.flatten()), ssh_1.flatten(),
                           (x_sparse, y_sparse), method='nearest')

# calculate gradients for geostrophic velocity
dSSH_dy, dSSH_dx = np.gradient(grid_ssh)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
U = -g / f * dSSH_dy
V = g / f * dSSH_dx

# visualization of SSH + velocity
plt.figure(figsize=(12, 6))

# plot SSH
plt.subplot(1, 2, 1)
plt.contourf(x_sparse, y_sparse, grid_ssh, levels=20, cmap='viridis')
plt.colorbar(label='SSH (m)')
plt.title('Interpolated Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# plot velocity field
plt.subplot(1, 2, 2)
plt.quiver(x_sparse, y_sparse, U, V)
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')

plt.tight_layout()
plt.show()

plt.figure()
plt.scatter (lon_1, lat_1, c=ssh_1, cmap='viridis', marker='o')
colorbar = plt.colorbar(fraction=0.05, pad=0.04,)
colorbar.set_ticks([-24, -19.5])

plt.show()

In [ ]:
# plot SSH anomaly + velocity field

# constants
g = 9.81  # acceleration due to gravity (m/s^2)
omega = 7.2921159e-5  # angular velocity of Earth (rad/s)

# aux function for mesh generation
def mesh(n):
    minval_lon = 153
    maxval_lon = 153.1
    minval_lat = -53.5
    maxval_lat = -53.4
    return np.meshgrid(np.linspace(minval_lon, maxval_lon, n),
                       np.linspace(minval_lat, maxval_lat, n))

# sparse input mesh for SSHA data
N_sparse = 38
x_sparse, y_sparse = mesh(N_sparse)

# interpolate SSHA onto dense grid
grid_ssha = interp.griddata((lon_1.flatten(), lat_1.flatten()), ssha_1.flatten(),
                           (x_sparse, y_sparse), method='linear')

# calculate gradients for geostrophic velocity
dSSHA_dy, dSSHA_dx = np.gradient(grid_ssha)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
U = -g / f * dSSHA_dy
V = g / f * dSSHA_dy

# visualization of SSHA and velocity
plt.figure(figsize=(12, 6))

# plot SSHA
plt.subplot(1, 2, 1)
plt.contourf(x_sparse, y_sparse, grid_ssha, levels=20, cmap='viridis')
plt.colorbar(label='SSHA (m)')
plt.title('Interpolated Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# plot velocity field
plt.subplot(1, 2, 2)
plt.quiver(x_sparse, y_sparse, U, V)
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')

plt.tight_layout()
plt.show()

plt.figure()
plt.scatter (lon_1, lat_1, c=ssha_1, cmap='viridis', marker='o')
colorbar = plt.colorbar(fraction=0.05, pad=0.04,)
colorbar.set_ticks([-24, -19.5])

plt.show()

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_2, lat_2)
p2 = np.array(np.meshgrid(lon_2, lat_2)).T.reshape(-1, 2)
v2 = ssh_2.flatten()
ssh2 = interp.griddata(p2, values, (lat_grid, lon_grid), method='linear')

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_3, lat_3)
p3 = np.array(np.meshgrid(lon_3, lat_3)).T.reshape(-1, 2)
v3 = ssh_3.flatten()
ssh3 = interp.griddata(p3, v3, (lat_grid, lon_grid), method='linear')

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_4, lat_4)
p4 = np.array(np.meshgrid(lon_4, lat_4)).T.reshape(-1, 2)
v4 = ssh_4.flatten()
ssh4 = interp.griddata(p4, v4, (lat_grid, lon_grid), method='linear')

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_5, lat_5)
p5 = np.array(np.meshgrid(lon_5, lat_5)).T.reshape(-1, 2)
v5 = ssh_5.flatten()
ssh5 = interp.griddata(p5, v5, (lat_grid, lon_grid), method='linear')

In [ ]:
# constants
g = 9.81           # acc due to grav (m/s^2)
omega = 7.2921e-5  # angular velocity of the Earth (rad/s)

In [ ]:
# calc ssh gradients
dH_dy, dH_dx = np.gradient(sea_surface_height_dense)

# calc coriolis parameter (convert lat to rads)
lat_rad = np.radians(lat_grid)
f = 2 * omega * np.sin(lat_rad)

In [ ]:
# calc geostrophic velocity components
u_velocity = -g / f * dH_dy  # eastward velocity component
v_velocity = g / f * dH_dx   # northward velocity component

In [ ]:
# plot sea surface velocity
plt.figure(figsize=(10, 6))
plt.quiver(lon_grid, lat_grid, u_velocity, v_velocity, scale=5, color='blue')
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.grid()
plt.show()

#### Vorticity Plots

In [ ]:
# open file
data = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc','r')

# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssh = data.variables ['ssh_karin'] [:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

# pass 155/433

In [ ]:
# set lon and lat limits 
lon_min, lon_max = 152, 155
lat_min, lat_max = -55, -50

# create mask
mask = (lon >= lon_min) & (lon <= lon_max) & (lat >= lat_min) & (lat <= lat_max) & (ssh <= 200000000) & (ssha <= 20000000) 

# apply mask to slice data
lon_1 = lon[mask]
lat_1 = lat[mask]
ssh_1 = ssh[mask] + hcorxover[mask]
ssha_1 = ssha[mask] + hcorxover[mask]

In [ ]:
longitude = np.array(lon_1)
latitude = np.array(lat_1)
sea_surface_height = np.array(ssh_1)

# create arrays to store uv, lon, & lat
uv_velocities = []
lon_list = []
lat_list = []

g = 9.81           # acc due to grav (m/s^2)
omega = 7.2921e-5  # angular velocity of the Earth (rad/s)

# for loop to collect uv values
for i in np.arange(152, 155, 0.1):
    for j in np.arange(-55, -50, 0.1):
        # Define the current box
        lon_min = i
        lon_max = i + 0.1
        lat_min = j
        lat_max = j + 0.1

        # filter data points in each box
        mask = (longitude >= lon_min) & (longitude < lon_max) & (latitude >= lat_min) & (latitude < lat_max)
        filtered_data = np.column_stack((longitude[mask], latitude[mask], sea_surface_height[mask]))
        
        # check if there are enough data points in each box + if the box contains nan
        if filtered_data.shape[0] < 3:  # at least 3 pts for plane fit
            continue
        if np.isnan(filtered_data).any():
            continue
        
        # least squares fitting
        A = np.c_[filtered_data[:, 0], filtered_data[:, 1], np.ones(filtered_data.shape[0])]
        C, _, _, _ = scipy.linalg.lstsq(A, filtered_data[:, 2])  # coefficient c

        # calculate d & e based on coefficient c 
        D = C[0] * (0.1 / 7000)
        E = C[1] * (0.1 / 11000)

        # calculate u and v
        f = 2 * omega * np.sin(np.radians(j)) 
        avg_f = np.mean(f)
        u = -g / avg_f * E
        v = g / avg_f * D

        if u > 5 or v > 5 or u < -5 or v < -5:
            continue

        # append uv, lon and lat
        uv_velocities.append([u, v])
        lon_list.append(lon_max)
        lat_list.append(lat_max)

# convert lists into 2d numpy array
uv_velocities_array = np.array(uv_velocities)
lon_array = np.array(lon_list)
lat_array = np.array(lat_list)

print("UV velocities array:")
for uv in uv_velocities_array:
    print(f"[ {uv[0]}, {uv[1]} ]")

In [ ]:
m = np.array(uv_velocities_array[:,0])
n = np.array(uv_velocities_array[:,1])

import matplotlib.axes as ax

plt.figure()
plt.scatter(lon_1, lat_1, s=8, c=ssh_1, cmap='viridis', marker='o')
plt.quiver(lon_array, lat_array, m, n, scale=40, label='velocity \n(between -1 and 1 m/s)')
cbar = plt.colorbar()
cbar.set_label('SSH (m)', rotation=270, labelpad=15)
plt.title('Velocities Over SSH (Pass 155)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.legend()
plt.show()

plt.figure()
plt.scatter(lon_1, lat_1, s=8, c=ssha_1, cmap='viridis', marker='o')
plt.quiver(lon_array, lat_array, m, n, scale=40, label='velocity \n(between -1 and 1 m/s)')
cbar = plt.colorbar()
cbar.set_label('SSH Anomaly (m)', rotation=270, labelpad=15)
plt.title('Velocities Over SSH Anomaly (Pass 155)')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')
plt.legend()
plt.show()

In [ ]:
u_velocities = m
v_velocities = n

# create grid for velocities
x = u_velocities
y = v_velocities
X, Y = np.meshgrid(x, y)

# calculate gradients
du_dy = np.gradient(u_velocities, axis=0)  # ∂u/∂y
dv_dx = np.gradient(v_velocities, axis=1)  # ∂v/∂x

# calculate vorticity
vorticity = dv_dx - du_dy

# plot vorticity field
plt.figure(figsize=(10, 6))
plt.contourf(X, Y, vorticity, levels=50, cmap='viridis')
plt.colorbar(label='Vorticity')
plt.title('Vorticity Field')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.quiver(X, Y, u_velocities, v_velocities, color='white', scale=10)  # Optional: add velocity vectors
plt.show()

In [ ]:
# check shapes of velocity arrays
print("u_velocities shape:", u_velocities.shape)
print("v_velocities shape:", v_velocities.shape)

if u_velocities.ndim == 1:
    u_velocities = u_velocities.reshape(-1, 1)  # Reshape to 2D if necessary
if v_velocities.ndim == 1:
    v_velocities = v_velocities.reshape(-1, 1)  # Reshape to 2D if necessary

# create a grid for velocities
x = np.linspace(0, 10, u_velocities.shape[1])
y = np.linspace(0, 10, u_velocities.shape[0])
X, Y = np.meshgrid(x, y)

# calculate gradients
du_dy = np.gradient(u_velocities, axis=0)  # ∂u/∂y
dv_dx = np.gradient(v_velocities, axis=1)  # ∂v/∂x

# calculate vorticity
vorticity = dv_dx - du_dy

# plot vorticity field
plt.figure(figsize=(10, 6))
plt.contourf(X, Y, vorticity, levels=50, cmap='viridis')
plt.colorbar(label='Vorticity')
plt.title('Vorticity Field')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.quiver(X, Y, u_velocities, v_velocities, color='white', scale=10)  # Optional: add velocity vectors
plt.show()

#### Sea Surface Height and Sea Surface Height Anomaly Velocity Fields for Singular SWOT Pass

In [ ]:
# open file
data = netCDF4.Dataset('SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc','r')

# gather variables
lat = data.variables['latitude'][:]
lon = data.variables['longitude'][:]
ssh = data.variables ['ssh_karin'] [:]
ssha = data.variables['ssha_karin'][:]
hcorxover = data.variables['height_cor_xover'][:]

ssh_1 = np.add(ssh, hcorxover)
ssha_1 = np.add(ssha, hcorxover)

ssh_mask = ssh_1 <= 200000000
ssha_mask = ssha_1 <= 20000000

ssh_new = ssh_1[ssh_mask]
ssha_new = ssha_1[ssha_mask]

# pass 155/433

In [ ]:
ssh_mask = ssh <= 200000000
ssha_mask = ssha <= 20000000

ssh_new = ssh[ssh_mask]
ssha_new = ssha[ssha_mask]

print(ssh_new)
print(ssha_new)

In [ ]:
# set lon and lat limits 
lon_min, lon_max = 154.5, 154.6
lat_min, lat_max = -52.7, -52.6

# create mask
mask = (lon >= lon_min) & (lon <= lon_max) & (lat >= lat_min) & (lat <= lat_max) & (ssh <= 200000000) & (ssha <= 20000000) 

# apply mask to slice data
lon_1 = lon[mask]
lat_1 = lat[mask]
ssh_1 = ssh[mask]
ssha_1 = ssha[mask]

In [ ]:
# constants
g = 9.81  # acceleration due to gravity (m/s^2)
omega = 7.2921159e-5  # angular velocity of Earth (rad/s)

# aux function for mesh generation
def mesh(n):
    minval_lon = 154.5
    maxval_lon = 154.6
    minval_lat = -52.7
    maxval_lat = -52.6
    return np.meshgrid(np.linspace(minval_lon, maxval_lon, n),
                       np.linspace(minval_lat, maxval_lat, n))

# sparse input mesh for SSH data
N_sparse = 38
x_sparse, y_sparse = mesh(N_sparse)

# interpolate SSH onto dense grid
grid_ssh = interp.griddata((lon_1.flatten(), lat_1.flatten()), ssh_1.flatten(),
                           (x_sparse, y_sparse), method='cubic')

# calculate gradients for geostrophic velocity

dSSH_dy, dSSH_dx = np.gradient(grid_ssh, (0.1/(N_sparse-1)), (0.1/(N_sparse-1)))

dSSH_dx_space = dSSH_dx*(0.1/7000)
dSSH_dy_space = dSSH_dy*(0.1/11000)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
U = -g / f * dSSH_dy_space
V = g / f * dSSH_dx_space

# visualization of SSH + velocity
plt.figure(figsize=(12, 6))

# plot SSH
plt.subplot(1, 2, 1)
plt.contourf(x_sparse, y_sparse, grid_ssh, levels=20, cmap='viridis')
plt.colorbar(label='SSH (m)')
plt.title('Interpolated Sea Surface Height')
plt.xlabel('Longitude (Degrees)')
plt.ylabel('Latitude (Degrees)')

# plot velocity field
plt.subplot(1, 2, 2)
plt.quiver(x_sparse, y_sparse, U, V)
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')

plt.tight_layout()
plt.show()

plt.figure()
plt.scatter (lon_1, lat_1, c=ssh_1, cmap='viridis', marker='o')
colorbar = plt.colorbar(fraction=0.05, pad=0.04,)
colorbar.set_ticks([-24, -19.5])

plt.show()

a = (y_sparse.shape)
print(a)

speed = np.sqrt((U**2)+(V**2))
plt.figure()
plt.scatter(x_sparse, y_sparse, c=speed, cmap='viridis', marker='o')
plt.colorbar()
plt.show()
print(speed)

In [ ]:
plt.figure()
plt.scatter(x_sparse, y_sparse, c=dSSH_dy_space, cmap='viridis', marker='o')
plt.colorbar()
plt.show()

In [ ]:
# constants
g = 9.81  # acceleration due to gravity (m/s^2)
omega = 7.2921159e-5  # angular velocity of Earth (rad/s)

# aux function for mesh generation
def mesh(n):
    minval_lon = 153
    maxval_lon = 153.1
    minval_lat = -53.5
    maxval_lat = -53.4
    return np.meshgrid(np.linspace(minval_lon, maxval_lon, n),
                       np.linspace(minval_lat, maxval_lat, n))

# sparse input mesh for SSHA data
N_sparse = len(lon_1)
x_sparse, y_sparse = mesh(N_sparse)

# interpolate SSHA onto dense grid
grid_ssha = interp.griddata((lon_1.flatten(), lat_1.flatten()), ssha_1.flatten(),
                           (x_sparse, y_sparse), method='linear')

# calculate gradients for geostrophic velocity
dSSHA_dy, dSSHA_dx = np.gradient(grid_ssha)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
U = -g / f * dSSHA_dy
V = g / f * dSSHA_dy

# visualization of SSHA and velocity
plt.figure(figsize=(12, 6))

# plot SSHA
plt.subplot(1, 2, 1)
plt.contourf(x_sparse, y_sparse, grid_ssha, levels=20, cmap='viridis')
plt.colorbar(label='SSHA (m)')
plt.title('Interpolated Sea Surface Height')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# plot velocity field
plt.subplot(1, 2, 2)
plt.quiver(x_sparse, y_sparse, U, V)
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')

plt.tight_layout()
plt.show()

plt.figure()
plt.scatter (lon_1, lat_1, c=ssha_1, cmap='viridis', marker='o')
colorbar = plt.colorbar(fraction=0.05, pad=0.04,)
colorbar.set_ticks([-24, -19.5])

plt.show()

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_4, lat_4)
p4 = np.array(np.meshgrid(lon_4, lat_4)).T.reshape(-1, 2)
v4 = ssh_4.flatten()
ssh4 = interp.griddata(p4, v4, (lat_grid, lon_grid), method='linear')

In [ ]:
lon_grid, lat_grid = np.meshgrid(lon_5, lat_5)
p5 = np.array(np.meshgrid(lon_5, lat_5)).T.reshape(-1, 2)
v5 = ssh_5.flatten()
ssh5 = interp.griddata(p5, v5, (lat_grid, lon_grid), method='linear')

In [ ]:
# constants
g = 9.81           # acc due to grav (m/s^2)
omega = 7.2921e-5  # angular velocity of the Earth (rad/s)

In [ ]:
# calc ssh gradients
dH_dy, dH_dx = np.gradient(sea_surface_height_dense)

# calc coriolis parameter (convert lat to rads)
lat_rad = np.radians(lat_grid)
f = 2 * omega * np.sin(lat_rad)

In [ ]:
# calc geostrophic velocity components
u_velocity = -g / f * dH_dy  # eastward velocity component
v_velocity = g / f * dH_dx   # northward velocity component

In [ ]:
# plot sea surface velocity
plt.figure(figsize=(10, 6))
plt.quiver(lon_grid, lat_grid, u_velocity, v_velocity, scale=5, color='blue')
plt.title('Sea Surface Velocity')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.grid()
plt.show()

In [ ]:
# sanity check
print(f)
print(U)
print(V)

In [ ]:
longitude = copy.deepcopy(lon_1)
latitude = copy.deepcopy(lat_1)
sea_surface_height = copy.deepcopy(ssh_1)

info = np.column_stack((longitude, latitude, sea_surface_height))

data = np.array(info)

lon_range = np.linspace(152.5, 152.6, 10)
lat_range = np.linspace(-53.6, -53.5, 10)
X, Y = np.meshgrid(lon_range, lat_range)
XX = X.flatten()
YY = Y.flatten()

order = 1  
if order == 1:
    # best-fit linear plane
    A = np.c_[data[:, 0], data[:, 1], np.ones(data.shape[0])]
    C, _, _, _ = scipy.linalg.lstsq(A, data[:, 2])  #coefficients
    
    # evaluate on grid
    Z = C[0] * X + C[1] * Y + C[2]

elif order == 2:
    # best-fit quadratic surface
    A = np.c_[np.ones(data.shape[0]), data[:, :2], np.prod(data[:, :2], axis=1), data[:, :2]**2]
    C, _, _, _ = scipy.linalg.lstsq(A, data[:, 2])  # coefficients
    
    # evaluate on grid
    Z = np.dot(np.c_[np.ones(XX.shape), XX, YY, XX * YY, XXc**2, YY**2], C).reshape(X.shape)

# plot points and fitted surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')  # Use add_subplot to create a 3D axis
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2)
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c='r', s=50)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
ax.set_zlabel('Sea Surface Height')
ax.axis('equal')
ax.axis('tight')
plt.show()

In [ ]:
longitude = copy.deepcopy(lon_1)
latitude = copy.deepcopy(lat_1)
sea_surface_height = copy.deepcopy(ssh_1)

# combine lat lon into a 2D array for fitting
info = np.column_stack((longitude, latitude, sea_surface_height))
data = np.array(info)

# regular grid covering the domain of data
lon_range = np.linspace(152.4, 152.5, 10)
lat_range = np.linspace(-53.6, -53.5, 10)
X, Y = np.meshgrid(lon_range, lat_range)

order = 1  
if order == 1:
    # best-fit linear plane
    A = np.c_[data[:, 0], data[:, 1], np.ones(data.shape[0])]
    C, _, _, _ = scipy.linalg.lstsq(A, data[:, 2])  # coefficients
    Z = C[0] * X + C[1] * Y + C[2]

# calculate derivatives
g = 9.81  # acceleration due to gravity (m/s^2)
f = 1e-4  # coriolis parameter (example value, adjust as necessary)


dSSH_dy, dSSH_dx = np.gradient(grid_ssh, (0.1/(N_sparse-1)), (0.1/(N_sparse-1)))

dSSH_dx_space = dSSH_dx*(0.1/7000)
dSSH_dy_space = dSSH_dy*(0.1/11000)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
# u = -g / f * dSSH_dy_space
# v = g / f * dSSH_dx_space

# create 3D scatter plot with fitted surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# plot fitted surface
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2, color='b')

# plot original data points
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c='r', s=50)

# add velocity vectors as arrows
arrow_density = 2  # adjust for fewer or more arrows
for i in range(0, len(longitude), arrow_density):
    ax.quiver(data[i, 0], data[i, 1], data[i, 2], u, v, 0, length=0.01, color='g')

# plot
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Sea Surface Height')
ax.set_title('3D Scatter Plot with Fitted Surface and Velocity Vectors')
ax.axis('equal')
plt.show()

In [ ]:
longitude = copy.deepcopy(lon_1)
latitude = copy.deepcopy(lat_1)
sea_surface_height = copy.deepcopy(ssh_1)

# calculate derivatives
g = 9.81  # acceleration due to gravity (m/s^2)
f = 1e-4  # coriolis parameter (example value, adjust as necessary)

dSSH_dy, dSSH_dx = np.gradient(grid_ssh, (0.1/(N_sparse-1)), (0.1/(N_sparse-1)))

dSSH_dx_space = dSSH_dx*(0.1/7000)
dSSH_dy_space = dSSH_dy*(0.1/11000)

N_sparse = len(lon_1)
x_sparse, y_sparse = mesh(N_sparse)

# calculate Coriolis parameter
f = 2 * omega * np.sin(np.radians(y_sparse))

# calculate velocities
# u = -g / f * dSSH_dy_space
# v = g / f * dSSH_dx_space

u = -g/f * E
v = g/f * D

# calculate velocity magnitudes
velocity_magnitude = np.sqrt(u**2 + v**2)

# create 3D scatter plot with fitted surface
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# plot fitted surface
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, alpha=0.2, color='b')

# plot original data points
sc = ax.scatter(data[:, 0], data[:, 1], data[:, 2], c='r', s=50)

# add velocity vectors as arrows
arrow_density = 2  # adjust for fewer or more arrows
for i in range(0, len(longitude), arrow_density):
    ax.quiver(data[i, 0], data[i, 1], data[i, 2], u[i], v[i], 0, length=0.01, color='g')

# create color bar for velocity magnitudes
norm = plt.Normalize(velocity_magnitude.min(), velocity_magnitude.max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])  
cbar = plt.colorbar(sm, ax=ax, pad=0.1)
cbar.set_label('Velocity Magnitude (m/s)')

# plot
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Sea Surface Height')
ax.set_title('3D Scatter Plot with Fitted Surface and Velocity Vectors')
ax.axis('equal')
plt.show()

In [ ]:
# sanity check
D = C[0]*(0.1/7000)
E = C[1]*(0.1/11000)

avg_f = np.mean(f)

u = -g / avg_f * E
v = g / avg_f * D


print(u,v)

In [ ]:
# create array to store u and v velocities
uv_velocities = []

# loop to collect u + v values
for i in np.arange(152, 154, 0.1):
    for j in np.arange(-54.0, -53.5, 0.1):
        f = 2 * omega * np.sin(np.radians(y_sparse))
        avg_f = np.mean(f)
        u = -g / avg_f * E
        v = g / avg_f * D

        # append u + v as a pair to list
        uv_velocities.append([u, v])

# convert into 2D numpy array
uv_velocities_array = np.array(uv_velocities)

# print
print("UV Velocities Array:")
for uv in uv_velocities_array:
    print(f"[ {uv[0]}, {uv[1]} ]")


### 3D Visualization

In [ ]:
# create an empty list to store data from each file
lat_all = []
lon_all = []
ssh_all = []
ssha_all = []
hcorxover_all = []

# all file names
file_names = ['SWOT_L2_LR_SSH_Expert_009_127_20240108T153645_20240108T162813_PGC0_01.nc', 
              'SWOT_L2_LR_SSH_Expert_009_155_20240109T153716_20240109T162844_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_183_20240110T153747_20240110T162915_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_254_20240113T043031_20240113T052159_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_282_20240114T043102_20240114T052231_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_310_20240115T043134_20240115T052302_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_338_20240116T043205_20240116T052333_PIC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_405_20240118T135903_20240118T145031_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_433_20240119T135934_20240119T145102_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_461_20240120T140005_20240120T145133_PGC0_01.nc',
              'SWOT_L2_LR_SSH_Expert_009_489_20240121T140036_20240121T145137_PGC0_01.nc']

# loop each file
for file_name in file_names:
    data = netCDF4.Dataset(file_name, 'r')
    lat = data.variables['latitude'][:]
    lon = data.variables['longitude'][:]
    ssh = data.variables['ssh_karin'][:]
    ssha = data.variables['ssha_karin'][:]
    hcorxover = data.variables['height_cor_xover'][:]

    lat_all.extend(lat)
    lon_all.extend(lon)
    ssh_all.extend(ssh)
    ssha_all.extend(ssha)
    hcorxover_all.extend(hcorxover)


longitude = np.array(lon_all)
latitude = np.array(lat_all)
ssh = np.array(ssh_all)
ssha = np.array(ssha_all)

lon_min, lon_max = 155, 160
lat_min, lat_max = -55, -52 

# create boolean mask based on the lat lon limits
mask = (longitude >= lon_min) & (longitude <= lon_max) & (latitude >= lat_min) & (latitude <= lat_max)

# mask to slice data
filtered_lon = longitude[mask]
filtered_lat = latitude[mask]
filtered_ssh = ssh[mask]
filtered_ssha = ssha[mask]

# sanity check
len(filtered_lon)

In [ ]:
np.max(filtered_ssh)
fill_ind = np.argwhere(filtered_ssh==2147483647.0)
filtered_ssh[fill_ind] = float('nan')

np.max(filtered_ssha)
fill_ind = np.argwhere(filtered_ssha==2147483647.0)
filtered_ssha[fill_ind] = float('nan')

In [ ]:
# fix random state for reproducibility
np.random.seed(19680801)


def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
    xs = filtered_lon
    ys = filtered_lat
    zs = filtered_ssh
    ax.scatter(xs, ys, zs, marker=m, c=filtered_ssh, cmap='viridis')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Sea Surface Height')

plt.show()

In [ ]:
np.random.seed(19680801)


def randrange(n, vmin, vmax):
    """
    Helper function to make an array of random numbers having shape (n, )
    with each number distributed Uniform(vmin, vmax).
    """
    return (vmax - vmin)*np.random.rand(n) + vmin

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

n = 100

# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
for m, zlow, zhigh in [('o', -50, -25), ('^', -30, -5)]:
    xs = filtered_lon
    ys = filtered_lat
    zs = filtered_ssha
    ax.scatter(xs, ys, zs, marker=m, c=filtered_ssha, cmap='viridis')

ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_zlabel('Sea Surface Height')

plt.show()